## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
import pandas as pd

In [2]:
tsv_file = open("archive//Restaurant_Reviews.tsv")
df = pd.read_csv(tsv_file, delimiter="\t")

In [3]:
df.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [4]:
###Drop Nan Values
df=df.dropna()


In [5]:
## Get the Independent Features

X=df.drop('Liked',axis=1)

In [6]:
## Get the Dependent features
y=df['Liked']

In [7]:
X.shape

(1000, 1)

In [8]:
y.shape

(1000,)

In [9]:
import tensorflow as tf

In [10]:
tf.__version__

'2.1.0'

In [11]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [12]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [13]:
messages=X.copy()

In [14]:
messages['Review'][1]

'Crust is not good.'

In [15]:
messages.reset_index(inplace=True)

In [16]:
import nltk
import re
from nltk.corpus import stopwords

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lesli\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['Review'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [19]:
corpus

['wow love place',
 'crust good',
 'tasti textur nasti',
 'stop late may bank holiday rick steve recommend love',
 'select menu great price',
 'get angri want damn pho',
 'honeslti tast fresh',
 'potato like rubber could tell made ahead time kept warmer',
 'fri great',
 'great touch',
 'servic prompt',
 'would go back',
 'cashier care ever say still end wayyy overpr',
 'tri cape cod ravoli chicken cranberri mmmm',
 'disgust pretti sure human hair',
 'shock sign indic cash',
 'highli recommend',
 'waitress littl slow servic',
 'place worth time let alon vega',
 'like',
 'burritto blah',
 'food amaz',
 'servic also cute',
 'could care less interior beauti',
 'perform',
 'right red velvet cake ohhh stuff good',
 'never brought salad ask',
 'hole wall great mexican street taco friendli staff',
 'took hour get food tabl restaur food luke warm sever run around like total overwhelm',
 'worst salmon sashimi',
 'also combo like burger fri beer decent deal',
 'like final blow',
 'found place acc

In [20]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[2787, 1803, 758],
 [4757, 367],
 [3651, 2033, 3657],
 [4290, 2432, 2989, 1944, 1912, 1342, 1776, 2601, 1803],
 [3686, 613, 2478, 159],
 [2628, 1095, 2457, 2296, 971],
 [3446, 4656, 732],
 [4169, 4879, 4220, 168, 4830, 4691, 3413, 4527, 2004, 1893],
 [62, 2478],
 [2478, 4289],
 [3293, 2395],
 [2775, 4137, 1806],
 [947, 4280, 201, 2434, 1301, 1093, 623, 1612],
 [1752, 3664, 2329, 2346, 2138, 2649, 2257],
 [245, 4665, 2465, 146, 2199],
 [3852, 4177, 1577, 4723],
 [2277, 2601],
 [896, 2583, 4568, 3293],
 [758, 949, 4527, 3979, 4810, 3750],
 [4879],
 [4968, 4633],
 [228, 2993],
 [3293, 1945, 1505],
 [168, 4280, 2987, 1083, 1739],
 [2813],
 [4481, 4648, 4939, 1514, 1485, 3705, 367],
 [4948, 2917, 2463, 1453],
 [1624, 3374, 2478, 4750, 565, 3105, 4636, 3772],
 [3477,
  1246,
  2628,
  228,
  853,
  1298,
  228,
  4980,
  854,
  4149,
  4612,
  2466,
  4879,
  3475,
  3172],
 [4448, 869, 714],
 [1945, 3828, 4879, 2713, 62, 52, 496, 3200],
 [4879, 4619, 3032],
 [880, 758, 3695, 168, 912],
 [3

### Embedding Representation

In [21]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2787 1803  758]
 [   0    0    0 ...    0 4757  367]
 [   0    0    0 ... 3651 2033 3657]
 ...
 [   0    0    0 ... 2775 4137 1806]
 [   0    0    0 ... 3334 2539 4527]
 [   0    0    0 ... 3477 4610 3039]]


In [22]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0, 2787, 1803,  758])

In [23]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
len(embedded_docs),y.shape

(1000, (1000,))

In [25]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [26]:
X_final.shape,y_final.shape

((1000, 20), (1000,))

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [34]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Train on 670 samples, validate on 330 samples
Epoch 1/10
670/670 [==============================] - 6s 9ms/sample - loss: 0.6933 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.4758
Epoch 2/10
670/670 [==============================] - 0s 324us/sample - loss: 0.6882 - accuracy: 0.5284 - val_loss: 0.6906 - val_accuracy: 0.4848
Epoch 3/10
670/670 [==============================] - 0s 321us/sample - loss: 0.6790 - accuracy: 0.5701 - val_loss: 0.6830 - val_accuracy: 0.5667
Epoch 4/10
670/670 [==============================] - 0s 390us/sample - loss: 0.6496 - accuracy: 0.7179 - val_loss: 0.6529 - val_accuracy: 0.6667
Epoch 5/10
670/670 [==============================] - 0s 502us/sample - loss: 0.5686 - accuracy: 0.7552 - val_loss: 0.6723 - val_accuracy: 0.5333
Epoch 6/10
670/670 [==============================] - 0s 520us/sample - loss: 0.4981 - accuracy: 0.8194 - val_loss: 0.5713 - val_accuracy: 0.7303
Epoch 7/10
670/670 [==============================] - 0s 527us/sample - loss: 0.

### Adding Dropout 

In [33]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [35]:
y_pred=model.predict_classes(X_test)

In [36]:
from sklearn.metrics import confusion_matrix

In [37]:
confusion_matrix(y_test,y_pred)

array([[140,  33],
       [ 50, 107]], dtype=int64)

In [38]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.7484848484848485